In [3]:
import numpy as np, pandas as pd
import surprise
from apyori import apriori

data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()

testSet = trainingSet.build_testset()

print(trainingSet.n_users)
print(trainingSet.n_items)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/victor/.surprise_data/ml-100k
943
1682


In [7]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
#df.head()

def create_movie_str(user_id):
    mask = (df['user_id'] == user_id) & (df['rating'] >= 4)
    return list(df[mask].movie_id.values)

users_list = df["user_id"].unique()

users_movies = []
for user_id in users_list:
    users_movies.append(create_movie_str(user_id))

print(users_movies)

[['393', '381', '655', '67', '306', '238', '663', '111', '25', '286', '692', '8', '428', '1118', '108', '382', '285', '1007', '411', '153', '1022', '845'], ['566', '148', '470', '281', '385', '588', '925', '322', '591', '550', '1277', '1253', '71', '257', '44', '117', '225', '988', '31', '939', '546', '100', '226', '300', '596', '1016', '79', '306', '829', '568', '332', '540', '98', '203', '291', '356', '684', '38', '477', '887', '330', '689', '159', '55', '1042', '177', '77', '147'], ['128', '80', '258', '510', '79', '511', '227', '173', '399', '186', '96', '117', '403', '999', '502', '435', '176', '222', '550', '393', '648', '118', '692', '226', '238', '455', '358', '208', '172', '230', '153', '154', '194', '195', '168', '204', '651', '187', '515', '85', '451', '840', '431', '202', '216', '184', '175', '24', '109', '523', '21', '201', '568', '181', '17', '144', '161', '712', '290', '430', '89', '228', '732', '4', '127', '62', '250', '50', '209', '526', '174', '385', '792', '68'], ['8

In [8]:
# https://medium.com/linkit-intecs/apriori-algorithm-in-data-mining-part-2-590d58e0998b
rules = apriori(users_movies, 
                min_support = 0.2,
                min_confidence = 0.3,
                min_lift = 2,
                min_length = 2)

# Store rules in result variable
results = list(rules)# See the items that were bought together with their support
results_list = []
for i in range(0, len(results)):
    results_list.append('RULE:\t' + str(results[i][0]) + '\nSUPPORT:\t' + str(results[i][1]))


print(len(results_list))

print(results_list[0])


10
RULE:	frozenset({'174', '172'})
SUPPORT:	0.2545068928950159


In [31]:
#problem 9.10.3 (not solved!)
from surprise import Reader, Dataset

print(df.head())

reader = Reader(rating_scale=(1.0, 1.0))
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

from surprise import KNNBasic

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)
trainingSet = data.build_full_trainset()

knn.fit(trainingSet)

testSet = trainingSet.build_testset()
predictions = knn.test(testSet)

print(predictions[7:8])
print(surprise.accuracy.rmse(predictions))

  user_id movie_id  rating          3
0     196      242     3.0  881250949
1     186      302     3.0  891717742
2      22      377     1.0  878887116
3     244       51     2.0  880606923
4     166      346     1.0  886397596
Computing the cosine similarity matrix...
Done computing similarity matrix.
[Prediction(uid='196', iid='238', r_ui=4.0, est=1.0, details={'actual_k': 39, 'was_impossible': False})]
RMSE: 2.7690
2.7689925965953757
